In [1]:
import pickle
import json
import re

In [2]:
def concat_conversation(data, labels, sep = ' | ', label_type = '_sent_c99_label'):
    conversations = []
    #summaries = []
    
    if sep != ' | ':
        sep += ' '
    
    for i in range(0, len(data)):
        if len(data[i].split('\r\n')) > 1:
            sentences = data[i].replace(" |", " ").split('<sep>')
        else:
            sentences = data[i].replace(" |", " ").split('<sep>')

        #summaries.append(data[i]['summary'].strip('\n').replace('\r\nt', ' '))

        if len(labels) > 1:
            
            if label_type == '_sent_c99_label':
                temp = ''
                temp += 'Speaker1: ' + sentences[0]
                for j in range(1, len(sentences)):
                    prefix = 'Speaker1: ' if j%2 == 0 else 'Speaker2: '
                    if labels[i][j] != labels[i][j-1]:
                        temp = temp + sep + prefix + sentences[j]
                    else:
                        temp = temp + ' ' + prefix + sentences[j]
                temp += ' | '
                conversations.append(temp)
            else:
                temp = ' | '
                temp += 'Speaker1: ' + sentences[0]
                for j in range(1, len(sentences)):
                    prefix = ' Speaker1: ' if j%2 == 0 else ' Speaker2: '
                    if labels[i][j] != labels[i][j-1]:
                        temp = temp + sep + prefix +  sentences[j]
                    else:
                        temp = temp + ' ' + prefix + sentences[j]
                conversations.append(temp)
        
        else:
            for i in range(len(sentences)):
                if i % 2 == 0:
                    sentences[i] = 'Speaker1: '+sentences[i]
                else:
                    sentences[i] = 'Speaker2: '+sentences[i]
                    
            if labels[0] == 1:
                conversations.append(' | ' + ' | '.join(sentences))
            elif labels[0] == 0:
                conversations.append(' | ' + ' '.join(sentences))

    return conversations
    

def transform_format(prefix, label_type = '_sent_c99_label'):
    with open(prefix, encoding = 'utf8') as file:
        data = file.readlines()
    if label_type != '_all' and label_type != '_none':
        with open(prefix + label_type +'.pkl', 'rb') as f:
            labels = pickle.load(f)
    elif label_type == '_all':
        labels = [1]
    elif label_type == '_none':
        labels = [0]
    
    cons = concat_conversation(data, labels, label_type)  

    with open(prefix + label_type +'.source', 'wt', encoding='utf-8') as source_file, open(prefix + label_type + '.target', 'wt', encoding='utf-8') as target_file:
        
        for i in range(0, len(cons)):
            article = cons[i]
            #abstract = sums[i]
            source_file.write(article + '\n')
            #target_file.write(abstract + '\n')

In [3]:
for u in ['1800train_100dev_100test.source']:
    # transform_format(u, '_all')
    # transform_format(u, '_none')
    transform_format(u, '_sent_trans_cons_label') 
    transform_format(u, '_sent_c99_label') 